# 功能一

In [1]:
import csv
from datetime import datetime
import shlex
import logging
import argparse
import string
from decimal import Decimal
from collections import defaultdict
import pandas as pd
import numpy as npa
from datetime import timedelta
import tqdm
import sys
import gc
from memory_profiler import profile

In [16]:
del all_dict["600166"]

In [17]:
%memit all_dict

peak memory: 108.55 MiB, increment: 0.00 MiB


In [3]:
# 函数
def dateToYYYYMMDD(rawDate):
    # date_ISO = datetime.strptime(rawDate, '%d/%m/%y').date()
    date_ISO = datetime.strptime(rawDate, '%Y%m%d').date()
    date_Str = date_ISO.strftime('%Y%m%d')
    return date_Str


def timetoHHMMSSF(rawTime):
    time_ISO = datetime.strptime(rawTime, '%H:%M:%S.%f')
    return time_ISO

In [4]:
def test10(all_dict,time_delta,fluc_range,fluc_money,in_volume,time,price,SID):
    time10=time-timedelta(minutes=time_delta)
    fluc_range=float(fluc_range)
    fluc_money=float(fluc_money)
    M_B=0
    M_S=0
    a=0
    p_max=price
    p_min=price
    for t in list(all_dict[SID].keys()):
        if t>=time10:
            for I in list(all_dict[SID][t].keys()):
                flag=all_dict[SID][t][I][2]
                if flag=="B":
                    v=all_dict[SID][t][I][0]
                    M_B+=all_dict[SID][t][I][1]*v
                    p_now=all_dict[SID][t][I][1]
                    if (p_now>p_max):
                        p_max=p_now
                    elif (p_now<p_min):
                        p_min=p_now
                    else:
                        pass
                elif flag=="S":
                    v=all_dict[SID][t][I][0]
                    M_S+=all_dict[SID][t][I][1]*v
                    p_now=all_dict[SID][t][I][1]
                    if (p_now>p_max):
                        p_max=p_now
                    elif (p_now<p_min):
                        p_min=p_now
                    else:
                        pass
                else:
                    pass
        else:
            del(all_dict[SID][t])
    p=round((p_max/p_min-1),6)
    M=round(M_B-M_S,2)
    if (p>=fluc_range)&(p<=0.1)&(M>=fluc_money):
        a=1
    else:
        pass
    return all_dict,a,p,M


In [13]:
% % memit
dataline = csv.DictReader(open("simple_data.csv"))
volume_dict = defaultdict(lambda: defaultdict(
    lambda: defaultdict(float)))  # 股票-时间-人-订单量
all_dict = defaultdict(lambda: defaultdict(
    lambda: defaultdict(lambda: None)))  # 股票-时间-人-（总订单量，价格,flag)
last_SID = defaultdict(lambda: None)  # 只显示一条
last_price = defaultdict(
    lambda: [0.001, timetoHHMMSSF("9:20:00.00")])  # 上一次达到条件时的股票对应的价格,时间

time_delta = 10
fluc_range = 0.07
fluc_money = 30000000
in_volume = 200000

for order in tqdm.tqdm_notebook(dataline):
    SID = order["SecurityID"]
    time = timetoHHMMSSF(order["TradTime"])
    price = float(order["TradPrice"])
    volume = float(order["TradVolume"])
    flag = order["TradeBSFlag"]
    SellID = order["TradeSellNo"]
    BuyID = order["TradeBuyNo"]
    money = float(order["TradeMoney"])
    list_SID = ['204001', '204002', '204003', '204004', '204005', '204006',
                '204007', '204014', '204028', '204091', '204182', '019654', '019658']
    ID = None
    if flag == "B":
        ID = BuyID
    elif flag == "S":
        ID = SellID
    else:
        continue
    if SID not in list_SID:
        volume_dict[SID][time][ID] += volume
        all_dict[SID][time][ID] = (volume_dict[SID][time][ID], price, flag)
    else:
        continue
    # 先将十分钟以外的数据删除
    t_max = list(all_dict[SID].keys())[-1]
    for t in list(all_dict[SID].keys()):
        if (t_max-t) >= timedelta(minutes=time_delta):
            del(all_dict[SID][t])
        else:
            break
#         gc.collect()
    if volume_dict[SID][time][ID] >= in_volume and last_SID[SID] != (time, ID) and (price/last_price[SID][0]-1) >= 0.07 and \
            (time-last_price[SID][1] >= timedelta(minutes=time_delta)):
        all_dict, a, p, M = test10(
            all_dict, time_delta, fluc_range, fluc_money, in_volume, time, price, SID)
        if a == 1:
            print("价格波动幅度：{},资金净流入：{},股票代码：{},时间：{},交易人员代码:{}".format(
                p, M, SID, str(time)[10:], ID))
            last_SID[SID] = (time, ID)
            last_price[SID] = [price, time]
    else:
        continue

<string>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


价格波动幅度：0.07013,资金净流入：118445884.96,股票代码：600166,时间： 09:36:39.910000,交易人员代码:1070760
价格波动幅度：0.092437,资金净流入：32598254.96,股票代码：601236,时间： 10:14:07.940000,交易人员代码:3419549

peak memory: 119.77 MiB, increment: 0.00 MiB


In [9]:
%load_ext memory_profiler

In [10]:
lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %conda  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %memit  %mkdir  %more  %mprun  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%memit  %

## 功能二

In [20]:
def test2s(all_dict):
    time2s=time-timedelta(seconds=2)
    M=0
    for t in list(all_dict[SID].keys()):
        if t>=time2s:
            for I in list(all_dict[SID][t].keys()):
                M=all_dict[SID][t][I]
        else:
                del(all_dict[SID][t])
    return all_dict,M

In [21]:
%%time
#11min 35s
RRL_Orderbook = csv.DictReader(open('simple_data.csv'))

import tqdm
money_dict = defaultdict(lambda: defaultdict(lambda: defaultdict(float)))#股票-时间-人-成交金额
_last = defaultdict(lambda: None)

for order in tqdm.tqdm_notebook(RRL_Orderbook):
    SID=order["SecurityID"]
    time=timetoHHMMSSF(order["TradTime"])
    price=float(order["TradPrice"])
    volume=float(order["TradVolume"])
    flag=order["TradeBSFlag"]
    SellID=order["TradeSellNo"]
    BuyID=order["TradeBuyNo"]
    money=float(order["TradeMoney"])

    if flag=="B":
        money_dict[SID][time][BuyID] += money
        money_dict,M=test2s(money_dict)
        if M>=50000000 and _last[SID] != (time, BuyID):
            print ("成交金额:{}, 股票代码:{}, 时间:{}, 交易人员代码:{}".format(M,SID,str(time)[10:],BuyID))
            _last[SID] = (time, BuyID)

<timed exec>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

Wall time: 1.93 s


## 功能三

In [22]:
def test60(all_dict,fluc_money,in_volume):
    fluc_money=float(fluc_money)
    M_B=0
    M_S=0
    a=0
    for t in list(all_dict[SID].keys()):
        for I in list(all_dict[SID][t].keys()):
            flag=all_dict[SID][t][I][2]
            if flag=="B":
                v=all_dict[SID][t][I][0]
                M_B+=all_dict[SID][t][I][1]*v
            elif flag=="S":
                v=all_dict[SID][t][I][0]
                M_S+=all_dict[SID][t][I][1]*v
            else:
                pass
    M=round(M_B-M_S,2)
    if (M>=fluc_money):
        a=1
    else:
        pass
    return all_dict,a,M

In [23]:
%%time
dataline=csv.DictReader(open("simple_data.csv"))
volume_dict = defaultdict(lambda: defaultdict(lambda: defaultdict(float)))#股票-时间-人-订单量
all_dict = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: None)))#股票-时间-人-（总订单量，价格,flag)
all_SID = defaultdict(lambda: None)#股票出现过就不再显示

fluc_money=50000000;in_volume=200000
time1=timetoHHMMSSF("9:30:00.00")
time2=timetoHHMMSSF("10:30:00.00")
for order in tqdm.tqdm_notebook(dataline):
    SID=order["SecurityID"]
    time=timetoHHMMSSF(order["TradTime"])
    volume=float(order["TradVolume"])
    price=float(order["TradPrice"])
    flag=order["TradeBSFlag"]
    SellID=order["TradeSellNo"]
    BuyID=order["TradeBuyNo"]
    money=float(order["TradeMoney"])
    #list_SID=['204001','204002','204003','204004','204005','204006','204007','204014','204028','204091','204182']
    if time>=time1 and all_SID[SID] != SID :
        ID=None
        if flag=="B":
            ID=BuyID
        elif flag=="S":
            ID=SellID
        else:
            continue
        volume_dict[SID][time][ID]+=volume
        all_dict[SID][time][ID]=(volume_dict[SID][time][ID],price,flag)
        if  volume_dict[SID][time][ID] >= in_volume : 
            all_dict,a,M=test60(all_dict,fluc_money,in_volume)
            if a==1:
                print ("资金净流入：{},股票代码：{},时间：{},交易人员代码:{}".format(M,SID,str(time)[10:],ID))
                all_SID[SID] = SID
    elif time >=time2:
        break
    else:
        continue

<timed exec>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

资金净流入：50008623.12,股票代码：600166,时间： 09:33:26.150000,交易人员代码:733881
Wall time: 2.46 s


## 功能四
### 就是功能一将参数改一下